In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Flatten
from copy import copy, deepcopy

In [ ]:
# HELPERS
def process(data):
    if not os.path.exists(os.path.join("graffiti_data",data)):
        raise FileNotFoundError("No such data folder. Try 'train', 'test', or 'valid'.")
    files = sorted([file for file in \
                    os.listdir(os.path.join("graffiti_data",data))\
                    if file[-4:] == ".jpg"])
    X = [load_img(os.path.join("graffiti_data",data,im)) for im in files]
    X = [img_to_array(im) for im in X]
    # X = [im.reshape((1, im.shape[0], im.shape[1], im.shape[2])) for im in X]
    X = np.array([preprocess_input(im) for im in X])
    classes = pd.read_csv(os.path.join('graffiti_data',data,'_classes.csv'))\
    .sort_values("filename")
    classes.insert(1,"pixvals",list(X))
    classes.reset_index(inplace=True)
    new_path = os.path.join("graffiti_data",f"{data}_data_full.csv")
    if os.path.exists(new_path):
        os.remove(new_path)
    classes.to_csv(new_path)
    return classes

In [ ]:
train = process("train")
train.shape

In [ ]:
train.head(1)

In [ ]:
x_train = np.array(list(thing for thing in train.values[:,2]))
y_train = train.values[:,-4:].astype(np.float32)

In [ ]:
x_train.shape

In [ ]:
# Load VGG16 without the top layer and with pooling
base_model = VGG16(include_top=False, input_shape=(400, 400, 3), pooling='max')

# Prevent training on the base model layers
# for layer in base_model.layers:
#     layer.trainable = False

# Properly connect new layers
x = Flatten()(base_model.output)  # Ensure the base model's output connects here
x = Dense(1024, activation='relu')(x)  # Connect this layer to the output of the Flatten layer
output = Dense(4, activation='softmax')(x)  # Connect this Dense layer to the previous Dense layer

# Finalize the model
model = Model(inputs=base_model.input, outputs=output)

for layer in model.layers[-8:]:
    layer.trainable = False
    
# Compile the model
model.compile(optimizer="Adam", loss="CategoricalCrossentropy", metrics=['accuracy'])

# Print the model structure to verify it is correct
model.summary()

In [ ]:
hist = model.fit(x_train, y_train, epochs=5)

In [ ]:
base_model.layers

In [ ]:
hist = model.fit(x_train, y_train, epochs=5)

In [ ]:
model.save("vgg_model_1")

In [ ]:
test = process("test")
test.shape

In [ ]:
x_test = np.array(list(thing for thing in test.values[:,2]))
y_test = test.values[:,-4:].astype(np.float32)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
def quantize(array):
    l = list(array)
    return np.array([round(i) for i in l])

In [ ]:
for i in range(len(y_pred)):
    y_pred[i] = quantize(y_pred[i])    

In [ ]:
def compare(a1,a2):
    if sum(a1==a2) == 4:
        return True
    return False

In [ ]:
truth_mat = []
for i in range(len(y_pred)):
    truth_mat.append(compare(y_pred[i],y_test[i]))

In [ ]:
print(f"Test Accuracy: {round(100* sum(truth_mat)/len(truth_mat),2)}%")